In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

In [2]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
split = 4

In [4]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_{split}.h5ad")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [5]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [6]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [7]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [8]:
adata_pred_test = adata_preds[adata_preds.obs["split"]=="test"]
adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds[adata_preds.obs["split"]=="ood"]
adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

/tmp/ipykernel_1715465/2350350615.py:2: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
/tmp/ipykernel_1715465/2350350615.py:18: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]


In [9]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [10]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [11]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [12]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [13]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [14]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9623616834481558,
 'decoded_ood_sinkhorn_div_1': 79.71715418497722,
 'decoded_ood_sinkhorn_div_10': 47.40566062927246,
 'decoded_ood_sinkhorn_div_100': 3.6434834798177085,
 'decoded_ood_e_distance': 6.70801098436969,
 'decoded_ood_mmd': 0.18034857014815012}

In [15]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7302497744560241,
 'decoded_test_e_distance': 53.07006995203459,
 'decoded_test_mmd_distance': 0.22581661939620973}

In [16]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9623616834481558,
 'decoded_ood_sinkhorn_div_1': 79.71715418497722,
 'decoded_ood_sinkhorn_div_10': 47.40566062927246,
 'decoded_ood_sinkhorn_div_100': 3.6434834798177085,
 'decoded_ood_e_distance': 6.70801098436969,
 'decoded_ood_mmd': 0.18034857014815012}

In [17]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -0.549099862575531,
 'encoded_ood_sinkhorn_div_1': 6.975557724634807,
 'encoded_ood_sinkhorn_div_10': 3.6264492670694985,
 'encoded_ood_sinkhorn_div_100': 2.7797206242879233,
 'encoded_ood_e_distance': 5.371199951601317,
 'encoded_ood_mmd': 0.102145670602719}

In [18]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.9444477458794912,
 'deg_ood_sinkhorn_div_1': 7.9162843227386475,
 'deg_ood_sinkhorn_div_10': 2.089114189147949,
 'deg_ood_sinkhorn_div_100': 1.7777109146118164,
 'deg_ood_e_distance': 3.4906960935067013,
 'deg_ood_mmd': 0.23377024630705515}

In [19]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.5236415028572082,
 'deg_test_sinkhorn_div_1': 25.43455644607544,
 'deg_test_sinkhorn_div_10': 16.593539867401123,
 'deg_test_sinkhorn_div_100': 15.402612400054931,
 'deg_test_e_distance': 30.63850786885545,
 'deg_test_mmd': 0.29712717711925507}

In [20]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7302497744560241,
 'decoded_test_e_distance': 53.07006995203459,
 'decoded_test_mmd_distance': 0.22581661939620973}

In [21]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9623616834481558,
 'decoded_ood_sinkhorn_div_1': 79.71715418497722,
 'decoded_ood_sinkhorn_div_10': 47.40566062927246,
 'decoded_ood_sinkhorn_div_100': 3.6434834798177085,
 'decoded_ood_e_distance': 6.70801098436969,
 'decoded_ood_mmd': 0.18034857014815012}

In [22]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.9444477458794912,
 'deg_ood_sinkhorn_div_1': 7.9162843227386475,
 'deg_ood_sinkhorn_div_10': 2.089114189147949,
 'deg_ood_sinkhorn_div_100': 1.7777109146118164,
 'deg_ood_e_distance': 3.4906960935067013,
 'deg_ood_mmd': 0.23377024630705515}

In [23]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7302497744560241,
 'decoded_test_e_distance': 53.07006995203459,
 'decoded_test_mmd_distance': 0.22581661939620973}

In [24]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -1.1632102525551038,
 'encoded_test_sinkhorn_div_1': 29.89208456993103,
 'encoded_test_sinkhorn_div_10': 26.153680267333986,
 'encoded_test_sinkhorn_div_100': 24.925610961914064,
 'encoded_test_e_distance': 49.57801048704666,
 'encoded_test_mmd': 0.22882987797260285}

In [25]:
ood_metrics_encoded

{'Cediranib+PCI-34051': {'r_squared': 0.6433114409446716,
  'sinkhorn_div_1': 5.025936126708984,
  'sinkhorn_div_10': 1.917597770690918,
  'sinkhorn_div_100': 1.1690521240234375,
  'e_distance': 2.1649807011694318,
  'mmd': 0.08687971},
 'Givinostat+Crizotinib': {'r_squared': -1.6947600841522217,
  'sinkhorn_div_1': 6.300126552581787,
  'sinkhorn_div_10': 2.864957809448242,
  'sinkhorn_div_100': 1.9458351135253906,
  'e_distance': 3.6940365457884345,
  'mmd': 0.09449098},
 'Givinostat+SRT2104': {'r_squared': -0.7927112579345703,
  'sinkhorn_div_1': 5.441862106323242,
  'sinkhorn_div_10': 2.038867950439453,
  'sinkhorn_div_100': 1.1772260665893555,
  'e_distance': 2.161713640046175,
  'mmd': 0.0863019},
 'Givinostat+Sorafenib': {'r_squared': -1.0975360870361328,
  'sinkhorn_div_1': 14.57806396484375,
  'sinkhorn_div_10': 11.237345695495605,
  'sinkhorn_div_100': 10.416598320007324,
  'e_distance': 20.660716980817995,
  'mmd': 0.17579031},
 'control+Givinostat': {'r_squared': -1.09412670

In [26]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/cpa"

In [27]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
